In [ ]:
from bs4 import BeautifulSoup
import requests as req
from time import sleep
import pandas as pd
import os
import time
from datetime import datetime as dt
from tqdm import tqdm
#from tqdm import tqdm


# configure pandas
#pd.set_option('display.max_rows', 500)

pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)

headers = {
    "User-Agent": (
        "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 "
        "(KHTML, like Gecko) Chrome/99.0.4844.51 Safari/537.36"
    )
}

city_map = {'New Orleans': 'New Orleans Saints',
 'Carolina': 'Carolina Panthers',
 'San Francisco': 'San Francisco 49ers',
 'Cincinnati': 'Cincinnati Bengals',
 'Philadelphia': 'Philadelphia Eagles',
 'Indianapolis': 'Indianapolis Colts',
 'Miami': 'Miami Dolphins',
 'Baltimore': 'Baltimore Ravens',
 'Washington': 'Washington Commanders',
 'Kansas City': 'Kansas City Chiefs',
 'LA Chargers': 'Los Angeles Chargers',
 'Minnesota': 'Minnesota Vikings',
 'Tennessee': 'Tennessee Titans',
 'Tampa Bay': 'Tampa Bay Buccaneers',
 'Denver': 'Denver Broncos',
 'Cleveland': 'Cleveland Browns',
 'Detroit': 'Detroit Lions',
 'NY Giants': 'New York Giants',
 'New England': 'New England Patriots',
 'LA Rams': 'Los Angeles Rams',
 'Las Vegas': 'Las Vegas Raiders',
 'Green Bay': 'Green Bay Packers',
 'Chicago': 'Chicago Bears',
 'Seattle': 'Seattle Seahawks',
 'Dallas': 'Dallas Cowboys',
 'Pittsburgh': 'Pittsburgh Steelers',
 'Jacksonville': 'Jacksonville Jaguars',
 'Arizona': 'Arizona Cardinals',
 'NY Jets': 'New York Jets',
 'Atlanta': 'Atlanta Falcons',
'Buffalo': 'Buffalo Bills'           
}

## Rank Data
____

In [124]:
date = "2022-10-26"
week = 7

dates_w_week = {
    "2022-11-02":8,
    "2022-10-26":7,
    "2022-10-19":6,
    "2022-10-12":5,
    "2022-10-05":4,
    "2022-09-28":3,
}

url = "https://www.teamrankings.com/nfl/stats/"
page = req.get(url, headers=headers).text
soup = BeautifulSoup(page, "html.parser")
pbar = tqdm(["points-per-game", "giveaways-per-game", "takeaways-per-game", "qb-sacked-per-game", "passing-yards-per-game", "rushing-yards-per-game"])
pd_list = []
for _date, week in dates_w_week.items():
    for end_point in pbar:
        time.sleep(.25)
        url = f"https://www.teamrankings.com/nfl/stat/{end_point}?date={_date}"
        page = req.get(url, headers=headers).text
        soup = BeautifulSoup(page, "html.parser")
        table = soup.find("table")
        df = pd.read_html(str(table))[0].replace("--",0)
        df["asof"] = _date
        df["week"] = week
        df["url"] = end_point
        file_name = end_point.replace('/', '_').replace("-","_") + "_week" + str(week)
        pbar.set_description(end_point)
        pd_list.append(df)
    
full_file_path = f"rank_data.csv"
df = pd.concat(pd_list)
df.to_csv(full_file_path)
print(df.head())

rushing-yards-per-game: 100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 6/6 [00:04<00:00,  1.47it/s]


   Rank          Team  2022  Last 3  Last 1  Home  Away  2021        asof  week              url
0     1   Kansas City  31.9    31.3    44.0  25.7  36.5  29.4  2022-11-02     8  points-per-game
1     2       Buffalo  29.0    29.7    27.0  35.3  24.3  29.8  2022-11-02     8  points-per-game
2     3  Philadelphia  28.0    27.0    35.0  28.5  27.3  25.5  2022-11-02     8  points-per-game
3     4       Seattle  26.3    27.7    27.0  21.5  31.0  23.2  2022-11-02     8  points-per-game
4     5     Baltimore  26.0    23.3    27.0  25.0  27.0  22.8  2022-11-02     8  points-per-game



### Spread and Game Data
___

In [ ]:
##### url = f"https://www.sportsoddshistory.com/nfl-game-season/?y=2022"
page = req.get(url, headers=headers).text
soup = BeautifulSoup(page, "html.parser")
tables = soup.find_all("table", attrs={"class":"soh1"})
pd_list = []
for i, table in enumerate(tables[2:]):
    df = pd.read_html(str(table))
    df = df[0]
    df.columns = df.iloc[-2]
    df = df.iloc[:-2,: ]
    df["week"] = i + 1
    pd_list.append(df)
df = pd.concat(pd_list)
print(df.head())
df.to_csv("/nfl_weekly_betting_summary.csv")

### Data Munging
____

In [243]:
df_games = pd.read_csv("nfl_weekly_betting_summary.csv")
df_ranks = pd.read_csv("rank_data.csv")

In [244]:
df_ranks["_stat"] = df_ranks.url
df_ranks["Stat"] = df_ranks._stat.str.split("-").str.join(" ").str.title()
df_ranks["Stat"] = df_ranks.Stat.str.replace("Per Game", "Pg").str.replace("Opponent", "Opp")\
.str.replace("Offensive", "Off")\
.str.replace("Defensive", "Def")\
.str.replace("Touchdowns", "Td")\
.str.replace("Rushing", "Run")\
.str.replace("Passing", "Pass")\
.str.replace("Interceptions", "Int")
df_ranks["full_team_name"] = df_ranks.Team.map(city_map)
df_ranks["rank_key"] = df_ranks.full_team_name + df_ranks._stat + df_ranks.week.apply(str)

In [245]:
df_games["fav_pts"] = df_games.Score.str.split(" ", expand=True)[1].str.split("-", expand=True)[0]
df_games["dog_pts"] = df_games.Score.str.split(" ", expand=True)[1].str.split("-", expand=True)[1]
df_fav = df_games[["Date", "Time (ET)", "Favorite", "Underdog", "Score", "week", "fav_pts"]].rename(columns={"Favorite":"Team", 
                                                                                                 "fav_pts":"team_pts_scored", 
                                                                                                 "Underdog":"Opp",
                                                                                                })

In [246]:
df_opp = df_games[["Date", "Time (ET)", "Favorite", "Underdog", "Score", "week", "dog_pts"]].rename(columns={"Underdog":"Team", 
                                                                                                 "dog_pts":"team_pts_scored",
                                                                                                 "Favorite":"Opp"
                                                                                                })
df_games = pd.concat([df_fav, df_opp])
df_games = df_games[df_games.week.gt(2)]


In [249]:
df_ranks

,Unnamed: 0,Rank,Team,2022,Last 3,Last 1,Home,Away,2021,asof,week,url,_stat,Stat,full_team_name,rank_key
0,0,1,Kansas City,31.9,31.3,44.0,25.7,36.5,29.4,2022-11-02,8,points-per-game,points-per-game,Points Pg,Kansas City Chiefs,Kansas City Chiefspoints-per-game8
1,1,2,Buffalo,29.0,29.7,27.0,35.3,24.3,29.8,2022-11-02,8,points-per-game,points-per-game,Points Pg,Buffalo Bills,Buffalo Billspoints-per-game8
2,2,3,Philadelphia,28.0,27.0,35.0,28.5,27.3,25.5,2022-11-02,8,points-per-game,points-per-game,Points Pg,Philadelphia Eagles,Philadelphia Eaglespoints-per-game8
3,3,4,Seattle,26.3,27.7,27.0,21.5,31.0,23.2,2022-11-02,8,points-per-game,points-per-game,Points Pg,Seattle Seahawks,Seattle Seahawkspoints-per-game8
4,4,5,Baltimore,26.0,23.3,27.0,25.0,27.0,22.8,2022-11-02,8,points-per-game,points-per-game,Points Pg,Baltimore Ravens,Baltimore Ravenspoints-per-game8
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1147,27,28,Las Vegas,80.0,80.0,96.0,80.0,80.0,95.6,2022-09-28,3,rushing-yards-per-game,rushing-yards-per-game,Run Yards Pg,Las Vegas Raiders,Las Vegas Raidersrushing-yards-per-game3
1148,28,29,Seattle,74.7,74.7,112.0,94.0,36.0,122.0,2022-09-28,3,rushing-yards-per-game,rushing-yards-per-game,Run Yards Pg,Seattle Seahawks,Seattle Seahawksrushing-yards-per-game3
1149,29,30,LA Rams,72.3,72.3,100.0,58.5,100.0,95.7,2022-09-28,3,rushing-yards-per-game,rushing-yards-per-game,Run Yards Pg,Los Angeles Rams,Los Angeles Ramsrushing-yards-per-game3
1150,30,31,Miami,64.0,64.0,41.0,53.0,86.0,92.2,2022-09-28,3,rushing-yards-per-game,rushing-yards-per-game,Run Yards Pg,Miami Dolphins,Miami Dolphinsrushing-yards-per-game3


In [252]:
df_games["rank_key"] = df_games.Team + "points-per-game" + df_games.week.apply(str)
df_games["team_points_pg"] = df_games.rank_key.map(df_ranks.set_index("rank_key")["2022"].to_dict())

df_games["rank_key"] = df_games.Opp + "points-per-game" + df_games.week.apply(str)
df_games["opp_points_pg"] = df_games.rank_key.map(df_ranks.set_index("rank_key")["2022"].to_dict())

df_games["rank_key"] = df_games.Team + "giveaways-per-game" + df_games.week.apply(str)
df_games["giveaways_pg"] = df_games.rank_key.map(df_ranks.set_index("rank_key")["2022"].to_dict())

df_games["rank_key"] = df_games.Team + "takeaways-per-game" + df_games.week.apply(str)
df_games["giveaways_pg"] = df_games.rank_key.map(df_ranks.set_index("rank_key")["2022"].to_dict())

df_games["rank_key"] = df_games.Opp + "giveaways-per-game" + df_games.week.apply(str)
df_games["takeaways_pg"] = df_games.rank_key.map(df_ranks.set_index("rank_key")["2022"].to_dict())

df_games["rank_key"] = df_games.Opp + "takeaways-per-game" + df_games.week.apply(str)
df_games["giveaways_pg"] = df_games.rank_key.map(df_ranks.set_index("rank_key")["2022"].to_dict())
